In [ ]:
from collections import OrderedDict
from datetime import datetime as dt
import os
import os.path as op
from pathlib import Path
import requests
from urllib.parse import unquote_plus


In [ ]:
with open("/Users/liang/Downloads/.furk_credentials") as f:
    ACCOUNT = next(f).strip().split(' = ')[-1]
    PWD = next(f).strip().split(' = ')[-1]

In [ ]:
s = requests.Session()

In [ ]:
r = s.get("https://www.furk.net/")

In [ ]:
login_payload = {
    'login': ACCOUNT,
    'pwd': PWD,
    'gigya_uid': '',
    'url': ''
}

In [ ]:
s.headers.update({'referer': 'https://www.furk.net/login'})
r = s.post("https://www.furk.net/api/login/login", params=login_payload)

In [ ]:
r_finished = s.get('https://www.furk.net/users/files/finished')

In [ ]:
r, r_finished

In [ ]:
r = s.post("https://www.furk.net/api/file/get")
j = r.json()
j

In [ ]:
from pyquery import PyQuery as pq

In [ ]:
dom = pq(r_finished.text)

In [ ]:
for html_elem in dom('div.list-group-item > h4.list-group-item-heading > a:first'):
    print(html_elem.attrib['href'], html_elem.text[:5])

In [ ]:
r_download = s.get("https://www.furk.net/" + "/df/xxx")

In [ ]:
root_li = pq(r_download.text)('#t_files_tree > ul > li')

In [ ]:
down_links = pq(root_li)('a.url-dl')
for elem in down_links:
    elem_dirs = pq(root_li)(elem).parents('.list-group-item')
    if elem_dirs:
        dir_names = [pq(elem_d).contents()[0].strip() for elem_d in elem_dirs]
        dir_names = [d for d in dir_names if d]
        if dir_names:
            print(' > '.join(dir_names), '>')

In [ ]:
pq(down_links[2]).attr('href')

In [ ]:
url = down_links[2].attrib['href']
unquote_plus(url.rsplit('/', 1)[1])

In [ ]:
[e.attrib['href'] for e in pq(r_download.text)("#t_files_tree a.url-dl")]

In [ ]:
dom = pq(r_download.text)

In [ ]:
list(filter(lambda d: 'title' in d.attrib and d.attrib['title'].startswith('http'), 
            dom(".col-md-4 br + div + small a")
))

In [ ]:
def make_uri_list(df_sublink):
    r_download = s.get("https://www.furk.net/" + df_sublink)
    html_elem = pq(r_download.text)("#plain_list")[0]
    if html_elem is None:
        raise ValueError("Parsing Error")
    return html_elem.text.splitlines()

In [ ]:
# with open("zip_mirrors.link", "w") as link_f:
#     download_links = [elem.attrib['href'] for elem in dom_download("small a.dl_link")]
#     link_f.write("\t".join(download_links) + '\n')

In [ ]:
TOP_DF = 12

w = IntProgressWidget()
w.max = TOP_DF
w.value = 0
ipydisplay(w)

r_finished = s.get('https://www.furk.net/users/files/finished')
finished_dom = pq(r_finished.text)
want_dfs = OrderedDict([
    (html_elem.text, html_elem.attrib['href'])
    for html_elem in finished_dom('div.tor-box h2 a:first')[:TOP_DF]
])
full_down_links = OrderedDict()
for df_title, df_sublink in want_dfs.items():
    w.description = "{:d} / {:d}".format(w.value + 1, TOP_DF)
    full_down_links[df_title] = make_uri_list(df_sublink)
    w.value += 1

In [ ]:
out_root = Path(os.path.expanduser("~/Downloads"), "aria_down")
out_root

In [ ]:
if not out_root.exists():
    out_root.mkdir()

In [ ]:
URI_PTH = out_root / 'down.uri'

In [ ]:
ARGUMENT_FMT = '''\
  dir={:s}
'''
with URI_PTH.open('w') as f:
    for df_title, df_links in full_down_links.items():
        if len(df_links) == 1:
            print(df_links[0], file=f)
        else:
            arg = ARGUMENT_FMT.format(df_title)
            for link in df_links:
                print(link, file=f)
                print(arg, file=f)

In [ ]:
furk_sublink = lambda sl='': 'https://www.furk.net/%s' % sl